# Reflection


In the context of LLM agent building, reflection refers to the process of prompting an LLM to observe its past steps (along with potential observations from tools/the environment) to assess the quality of the chosen actions.
This is then used downstream for things like re-planning, search, or evaluation.

![Reflection](./img/reflection.png)

This notebook demonstrates a very simple form of reflection in LangGraph.

#### Prerequisites

We will be using a basic agent with a search tool here.

In [1]:
%pip install -U --quiet  langgraph langchain-fireworks
%pip install -U --quiet tavily-python

In [2]:
import getpass
import os


def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var)


# Optional: Configure tracing to visualize and debug the agent
_set_if_undefined("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Reflection"

_set_if_undefined("FIREWORKS_API_KEY")

LANGCHAIN_API_KEY ········
FIREWORKS_API_KEY ········


## Generate

For our example, we will create a "5 paragraph essay" generator. First, create the generator:


In [3]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_fireworks import ChatFireworks

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm = ChatFireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    max_tokens=32768
)
generate = prompt | llm

In [4]:
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

Title: The Eternal Relevance of The Little Prince in Modern Childhood

Introduction:
Antoine de Saint-Exupéry's The Little Prince is a timeless novella that has captured the hearts and minds of children and adults alike for over seven decades. Its enduring charm and profound wisdom have transcended generations, making it a classic staple in childhood literature. This essay explores the reasons why The Little Prince remains relevant in modern childhood.

First Paragraph:
One of the primary reasons for The Little Prince's relevance is its exploration of themes that resonate with children today. The story addresses universal aspects of childhood, such as the struggle to understand the world, the desire for friendship and love, and the pain of loss and loneliness. The Little Prince's encounters with various grown-ups, each representing different facets of adult absurdity, mirror the confusion and disillusionment children experience as they grow and navigate their way through a complex worl

### Reflect

In [5]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [6]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

Essay Critique and Recommendations:

Title: The Eternal Relevance of The Little Prince in Modern Childhood

Introduction:
The introduction effectively sets the stage for the essay by providing background information on Antoine de Saint-Exupéry's The Little Prince and its enduring popularity.

First Paragraph:
The first paragraph does an excellent job of discussing the relevance of the novella's themes to modern childhood. However, it would be beneficial to provide specific examples from the book to strengthen the argument and make it more engaging for the reader. Consider adding direct quotes or specific scenes that illustrate the points being made.

Second Paragraph:
The second paragraph highlights the values promoted by The Little Prince and their relevance to modern childhood. This paragraph is well-written and engaging, but it could benefit from more development. Consider expanding on the connection between the story's values and the challenges faced by children today. Additionally

### Repeat

And... that's all there is too it! You can repeat in a loop for a fixed number of steps, or use an LLM (or other check) to decide when the finished product is good enough.

In [7]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

Title: The Eternal Relevance of The Little Prince in Modern Childhood

Introduction:
The introduction effectively sets the stage for the essay by providing background information on Antoine de Saint-Exupéry's The Little Prince and its enduring popularity.

First Paragraph:
The first paragraph has been revised to include specific examples from the book, strengthening the argument and making it more engaging for the reader. Direct quotes and scenes from the novella have been added to illustrate the points being made.

Second Paragraph:
The second paragraph has been expanded to better develop the connection between the story's values and the challenges faced by children today. Real-world examples and anecdotes have been incorporated to further illustrate the importance of these values.

Third Paragraph:
The third paragraph has been updated to discuss recent environmental issues and the relevance of The Little Prince's message in the current context. This strengthens the argument for the s

## Define graph

Now that we've shown each step in isolation, we can wire it up in a graph.

In [8]:
from typing import Annotated, List, Sequence
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]


async def generation_node(state: State) -> State:
    return {"messages": [await generate.ainvoke(state['messages'])]}


async def reflection_node(state: State) -> State:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [state['messages'][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state['messages'][1:]
    ]
    res = await reflect.ainvoke(translated)
    # We treat the output of this as human feedback for the generator
    return {"messages": [HumanMessage(content=res.content)]}


builder = StateGraph(State)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_edge(START, "generate")


def should_continue(state: List[BaseMessage]):
    if len(state["messages"]) > 6:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

In [9]:
async for event in graph.astream({
    "messages": [
        HumanMessage(
            content="Generate an essay on the topicality of The Little Prince and its message in modern life"
        )
    ],
}):
    print(event)
    print("---")

{'generate': {'messages': [AIMessage(content='Title: The Little Prince: A Topical Allegory for Modern Life\n\nIntroduction:\nAntoine de Saint-Exupéry\'s "The Little Prince" is a classic novella that has captured the hearts of millions since its publication in 1943. While it might be easy to dismiss this work as a children\'s story, its profound themes and timeless message make it a relevant and topical piece in modern life. This essay will explore the allegorical nature of "The Little Prince" and discuss how its message can be applied to the complexities of the modern world.\n\nBody Paragraph 1 - The Allegory of the Little Prince:\n"The Little Prince" is an allegorical tale that explores various aspects of the human condition through its whimsical characters and situations. The Little Prince himself represents innocence, curiosity, and the importance of human connection. As the story unfolds, readers encounter different characters that symbolize various aspects of adult life, such as v

In [12]:
ChatPromptTemplate.from_messages(event["generate"]["messages"]).pretty_print()

================================== Ai Message ==================================

Title: The Little Prince: A Topical Allegory for Modern Life

Introduction:
In the vast expanse of the universe, a young prince travels from planet to planet, uncovering the complexities of the human condition. Antoine de Saint-Exupéry's "The Little Prince" is a timeless tale that transcends generations, offering profound insights into the nature of relationships, materialism, and the human experience. This essay will delve into the allegorical nature of "The Little Prince," explore its relevance in modern life, and discuss the story's critique of contemporary society.

Body Paragraph 1 - The Allegory of the Little Prince:
The Little Prince encounters various inhabitants of different planets, each representing a distinct aspect of adult life. For instance, the king symbolizes authority, while the businessman embodies materialism. These allegorical representations resonate with contemporary readers, as the

## Conclusion

Now that you've applied reflection to an LLM agent, I'll note one thing: self-reflection is inherently cyclic: it is much more effective if the reflection step has additional context or feedback (from tool observations, checks, etc.). If, like in the scenario above, the reflection step simply prompts the LLM to reflect on its output, it can still benefit the output quality (since the LLM then has multiple "shots" at getting a good output), but it's less guaranteed.
